In [36]:
import pandas as pd
import matplotlib
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, f1_score, confusion_matrix,r2_score, mean_squared_error
from sklearn.svm import SVR
from xgboost import XGBClassifier, XGBRegressor
from sklearn.svm import LinearSVR
from sklearn.compose import ColumnTransformer
from IPython.display import display
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV


In [2]:
og_data = pd.read_csv(r'data/Warehouse_and_Retail_Sales.csv')

In [3]:
## Functions

def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]: 
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Autumn'
    
def onehoencon(model_df):
    ohe = OneHotEncoder()
    df_numerical = model_df.select_dtypes(include='number')  
    df_features = model_df.select_dtypes(include='object')
    #print(df_features)              
    feature_array = ohe.fit_transform(df_features).toarray() #One-hot enconding
    #print(feature_array.shape)
    cat_features = pd.DataFrame(feature_array, columns=ohe.get_feature_names_out()) #Get columns names in the df 
    print(cat_features.info())
    print(df_numerical.info())
    #print(ohe.get_feature_names_out())
    ohe_df = pd.concat([cat_features, df_numerical], axis = 1) # Concatenate with numerical
    #print(ohe_df)
    return ohe_df    

In [4]:
og_data.info()
og_data.head(5) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128355 entries, 0 to 128354
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   YEAR              128355 non-null  int64  
 1   MONTH             128355 non-null  int64  
 2   SUPPLIER          128331 non-null  object 
 3   ITEM CODE         128355 non-null  object 
 4   ITEM DESCRIPTION  128355 non-null  object 
 5   ITEM TYPE         128354 non-null  object 
 6   RETAIL SALES      128355 non-null  float64
 7   RETAIL TRANSFERS  128355 non-null  float64
 8   WAREHOUSE SALES   128355 non-null  float64
dtypes: float64(3), int64(2), object(4)
memory usage: 8.8+ MB


,YEAR,MONTH,SUPPLIER,ITEM CODE,ITEM DESCRIPTION,ITEM TYPE,RETAIL SALES,RETAIL TRANSFERS,WAREHOUSE SALES
0,2017,4,ROYAL WINE CORP,100200,GAMLA CAB - 750ML,WINE,0.0,1.0,0.0
1,2017,4,SANTA MARGHERITA USA INC,100749,SANTA MARGHERITA P/GRIG ALTO - 375ML,WINE,0.0,1.0,0.0
2,2017,4,JIM BEAM BRANDS CO,10103,KNOB CREEK BOURBON 9YR - 100P - 375ML,LIQUOR,0.0,8.0,0.0
3,2017,4,HEAVEN HILL DISTILLERIES INC,10120,J W DANT BOURBON 100P - 1.75L,LIQUOR,0.0,2.0,0.0
4,2017,4,ROYAL WINE CORP,101664,RAMON CORDOVA RIOJA - 750ML,WINE,0.0,4.0,0.0


In [5]:
og_data.dropna(inplace= True)
og_data =og_data.reset_index(drop=True)



In [6]:
#Feature engineering quarter and season
df1 = og_data

# We use floor function to get quarter
df1['QUARTER']= (df1['MONTH'] - 1) // 3 + 1
# We use the function get_season and apply
df1['SEASON']= df1['MONTH'].apply(get_season)

df1

,YEAR,MONTH,SUPPLIER,ITEM CODE,ITEM DESCRIPTION,ITEM TYPE,RETAIL SALES,RETAIL TRANSFERS,WAREHOUSE SALES,QUARTER,SEASON
0,2017,4,ROYAL WINE CORP,100200,GAMLA CAB - 750ML,WINE,0.00,1.0,0.0,2,Spring
1,2017,4,SANTA MARGHERITA USA INC,100749,SANTA MARGHERITA P/GRIG ALTO - 375ML,WINE,0.00,1.0,0.0,2,Spring
2,2017,4,JIM BEAM BRANDS CO,10103,KNOB CREEK BOURBON 9YR - 100P - 375ML,LIQUOR,0.00,8.0,0.0,2,Spring
3,2017,4,HEAVEN HILL DISTILLERIES INC,10120,J W DANT BOURBON 100P - 1.75L,LIQUOR,0.00,2.0,0.0,2,Spring
4,2017,4,ROYAL WINE CORP,101664,RAMON CORDOVA RIOJA - 750ML,WINE,0.00,4.0,0.0,2,Spring
...,...,...,...,...,...,...,...,...,...,...,...
128325,2018,2,LEGENDS LTD,99753,DUTCHESS DE BOURGOGNE NR - 750ML,BEER,0.00,0.0,2.0,1,Winter
128326,2018,2,COASTAL BREWING COMPANY LLC,99813,DOMINION OAK BARREL STOUT 1/2K,KEGS,0.00,0.0,2.0,1,Winter
128327,2018,2,ANHEUSER BUSCH INC,9997,HOEGAARDEN 4/6NR - 12OZ,BEER,66.46,59.0,212.0,1,Winter
128328,2018,2,COASTAL BREWING COMPANY LLC,99970,DOMINION OAK BARREL STOUT 4/6 NR - 12OZ,BEER,9.08,7.0,35.0,1,Winter


In [7]:
# Sum all sales columns
df1['SALES']= df1['RETAIL SALES'] + df1['RETAIL TRANSFERS'] + df1['WAREHOUSE SALES']

df1.drop(columns= ['RETAIL SALES', 'RETAIL TRANSFERS', 'WAREHOUSE SALES'], inplace=True)


In [8]:
df1['ITEM TYPE'].value_counts()


WINE            79077
LIQUOR          28397
BEER            16425
KEGS             3638
NON-ALCOHOL       628
STR_SUPPLIES      106
REF                35
DUNNAGE            24
Name: ITEM TYPE, dtype: int64

In [9]:
# Lets drop item description and item code 
df1.drop(columns= ['ITEM DESCRIPTION', 'ITEM CODE'], inplace=True)

In [10]:
## I cannot apply one-hot straight away so i will apply some dimensionality reduction technique
#PCA
# Perform PCA
#pca = PCA(n_components=2)
#_pca = pca.fit_transform(df1)

# # Print the explained variance ratio
# print("Explained Variance Ratio:", pca.explained_variance_ratio_)

# # Plot the original data
# plt.scatter(X[:, 0], X[:, 1], label='Original Data')

# # Plot the transformed data after PCA
# plt.scatter(X_pca[:, 0], X_pca[:, 1], label='PCA Transformed Data')

# plt.xlabel('Feature 1')
# plt.ylabel('Feature 2')
# plt.title('PCA Example')
# plt.legend()
# plt.show()
df1

,YEAR,MONTH,SUPPLIER,ITEM TYPE,QUARTER,SEASON,SALES
0,2017,4,ROYAL WINE CORP,WINE,2,Spring,1.00
1,2017,4,SANTA MARGHERITA USA INC,WINE,2,Spring,1.00
2,2017,4,JIM BEAM BRANDS CO,LIQUOR,2,Spring,8.00
3,2017,4,HEAVEN HILL DISTILLERIES INC,LIQUOR,2,Spring,2.00
4,2017,4,ROYAL WINE CORP,WINE,2,Spring,4.00
...,...,...,...,...,...,...,...
128325,2018,2,LEGENDS LTD,BEER,1,Winter,2.00
128326,2018,2,COASTAL BREWING COMPANY LLC,KEGS,1,Winter,2.00
128327,2018,2,ANHEUSER BUSCH INC,BEER,1,Winter,337.46
128328,2018,2,COASTAL BREWING COMPANY LLC,BEER,1,Winter,51.08


In [11]:
## if items are ordinal then you could opt to use label encoding
## Apply one-hot
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
ohe_df = onehoencon(df1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128330 entries, 0 to 128329
Columns: 345 entries, SUPPLIER_8 VINI INC to SEASON_Winter
dtypes: float64(345)
memory usage: 337.8 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128330 entries, 0 to 128329
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   YEAR     128330 non-null  int64  
 1   MONTH    128330 non-null  int64  
 2   QUARTER  128330 non-null  int64  
 3   SALES    128330 non-null  float64
dtypes: float64(1), int64(3)
memory usage: 3.9 MB
None


In [12]:
ohe_df.tail(2)

,SUPPLIER_8 VINI INC,SUPPLIER_A HARDY USA LTD,SUPPLIER_A I G WINE & SPIRITS,SUPPLIER_A VINTNERS SELECTIONS,SUPPLIER_A&E INC,SUPPLIER_A&W BORDERS LLC,SUPPLIER_ADAMBA IMPORTS INTL,SUPPLIER_AIKO IMPORTERS INC,SUPPLIER_ALLAGASH BREWING COMPANY,SUPPLIER_ALLIED IMPORTERS USA LTD,SUPPLIER_ALTITUDE SPIRITS INC,SUPPLIER_AMERICAN BEVERAGE CORPORATION,SUPPLIER_AMERICAN BEVERAGE MARKETERS,SUPPLIER_AMERICAN FIDELITY TRADING,SUPPLIER_AMERICAN VINTAGE BEVERAGE INC,SUPPLIER_ANHEUSER BUSCH INC,SUPPLIER_ARCHER ROOSE LLC,SUPPLIER_AREL GROUP WINE & SPIRITS,SUPPLIER_ARIS A ZISSIS,SUPPLIER_ARTISANS & VINES LLC,SUPPLIER_ASAHI BEER USA INC,SUPPLIER_ATLANTIC WINE & SPIRITS,SUPPLIER_ATLAS BREW WORKS LLC,SUPPLIER_AW DIRECT LLC,SUPPLIER_AZIZ SHAFI TANNIC TONGUE,SUPPLIER_BACARDI USA INC,SUPPLIER_BACCHUS IMPORTERS LTD,SUPPLIER_BACKUP BEVERAGE,SUPPLIER_BANFI PRODUCTS CORP,SUPPLIER_BANVILLE & JONES WINE MERCHANTS,SUPPLIER_BARON FRANCOIS LTD,SUPPLIER_BARREL ONE INC,SUPPLIER_BASIGNANI WINERY,SUPPLIER_BINDING BRAUEREI USA INC,SUPPLIER_BLACK ANKLE VINEYARDS LLC,SUPPLIER_BOND DISTRIBUTING CO,SUPPLIER_BOORDY VINEYARDS,SUPPLIER_BORVIN BEVERAGE,SUPPLIER_BOSTON BEER CORPORATION,SUPPLIER_BOUTIQUE VINEYARDS LLC,SUPPLIER_BRONCO WINE COMPANY,SUPPLIER_BROWN-FORMAN BEVERAGES WORLDWIDE,SUPPLIER_BUCK DISTRIBUTING COMPANY INC,SUPPLIER_BULLSHINE DISTILLERY LLC,SUPPLIER_C MONDAVI & SONS,SUPPLIER_CABERNET CORPORATION,SUPPLIER_CAFFO BEVERAGES INC,SUPPLIER_CALVERT BREWING COMPANY,SUPPLIER_CAMPARI AMERICA LLC,SUPPLIER_CANTINIERE IMPORTS & DISTRIBUTING INC,SUPPLIER_CAPE STARZ WINE LLC,SUPPLIER_CASA CHIRICO LLC,SUPPLIER_CASTLE BRANDS USA CORP,SUPPLIER_CELICAR LLC DBA TWIN VALLEY DISTILLERS,SUPPLIER_CHADDSFORD WINERY LTD,SUPPLIER_CHARLES JACQUIN ET CIE INC,SUPPLIER_CHATEAU DIANA LLC,SUPPLIER_CHEF SOUS LLC,SUPPLIER_CHEVAL QUANCARD,SUPPLIER_CLASSIC WINE IMPORTS INC,SUPPLIER_CLASSIC WINE IMPORTS INC DBA VISION WINE,SUPPLIER_CLIPPER CITY BREWING CO,SUPPLIER_COASTAL BREWING COMPANY LLC,SUPPLIER_COMETE WINES LLC,SUPPLIER_CONSTANTINE WINES INC,SUPPLIER_CONSTELLATION BRANDS,SUPPLIER_COPPER CANE LLC,SUPPLIER_CRAFT BREW ALLIANCE INC,SUPPLIER_CRAFT WINE & SPIRITS OF MARYLAND LLC,SUPPLIER_CROWN IMPORTS,SUPPLIER_D'AQUINO ITALIAN IMPORTING CO INC,SUPPLIER_DANGEROUS WINE GROUP LLC,SUPPLIER_DC BRAU BREWING LLC,SUPPLIER_DCD ENTERPRISES LLC,SUPPLIER_DELICATO FAMILY VINEYARDS,SUPPLIER_DELMAR BREWING COMPANY LLC,SUPPLIER_DESCHUTES BREWERY INC,SUPPLIER_DEUTSCH FAMILY WINE & SPIRITS,SUPPLIER_DIAGEO NORTH AMERICA INC,SUPPLIER_DIONYSOS IMPORTS INC,SUPPLIER_DISARONNO INTERNATIONAL LLC,SUPPLIER_DMV DISTRIBUTING LLC,SUPPLIER_DOGFISH HEAD CRAFT BREWERY,SUPPLIER_DOGFISH HEAD DISTILLERY LLC,SUPPLIER_DOMAINE SELECT WINE & SPIRITS LLC,SUPPLIER_DON SEBASTIANI & SONS,SUPPLIER_DOPS INC,SUPPLIER_DRAGON DISTILLERY LLC,SUPPLIER_DREYFUS ASHBY INC,SUPPLIER_DUCKHORN WINE COMPANY,SUPPLIER_DUCLAW BREWERY LLC,SUPPLIER_DUGGANS DISTILLERS PROD,SUPPLIER_DUVEL MOORTGAT USA LTD,SUPPLIER_Default,SUPPLIER_E & J GALLO WINERY,SUPPLIER_E M D SALES INC,SUPPLIER_EASTERN LIQUOR DISTRIBUTORS INC,SUPPLIER_ELITE WINES IMPORTS,SUPPLIER_ELK RUN VINEYARDS,SUPPLIER_EVEREST SPIRITS LLC,SUPPLIER_EXCELSIOR WINE COMPANY LLC,SUPPLIER_FAR NIENTE WINERY INC,SUPPLIER_FAROPIAN SPIRITS LTD,SUPPLIER_FERRARI CARANO WINERY,SUPPLIER_FETZER VINEYARDS,SUPPLIER_FIFTH GENERATION INC,SUPPLIER_FIORE WINERY,SUPPLIER_FIVE GRAPES LLC,SUPPLIER_FLAVOR SEEKERS LLC,SUPPLIER_FLYING DOG BREWERY LLLP,SUPPLIER_FN CELLARS LLC,SUPPLIER_FOLEY FAMILY WINES INC,SUPPLIER_FOWLES WINE USA INC,SUPPLIER_FRANCIS COPPOLA WINERY LLC,SUPPLIER_FRANK LIN MD,SUPPLIER_FREDERICK P WINNER LTD,SUPPLIER_FREDERICK WILDMAN & SONS,SUPPLIER_FREE RUN WINE MERCHANTS LLC,SUPPLIER_FREELANCE WINES LLC,SUPPLIER_FREIXENET USA,SUPPLIER_FRITZ IMPORTS LLC,SUPPLIER_FRONT PORCH MICRO WINERY LLC,SUPPLIER_FRUIT OF THE VINES INC,SUPPLIER_FX MATT BREWING CO,SUPPLIER_GAMBRINUS,SUPPLIER_GATOR RON'S LLC,SUPPLIER_GEORGIAN HOUSE OF GREATER WASHINGTON,"SUPPLIER_GF WINES, LLC DBA GARY FARRELL WINERY",SUPPLIER_GLOBAL OCEAN AND

In [13]:
# Lets split the dataset to train and test
og_data['YEAR'].value_counts()
# We see that 2017 is the 80 % of the dataset so i will use year to split 80 to 20
y = ohe_df[['SALES', 'YEAR']]
X = ohe_df.drop(columns=['SALES'])

X_train = X[X['YEAR'] == 2017]
X_test = X[X['YEAR'] == 2018]
y_train = y[y['YEAR'] == 2017]
y_test = y[y['YEAR'] == 2018]
# define standard scaler
scaler = StandardScaler()
#Notice the slight difference between fit transform and transform for train and test
#Drop year from y
y = y.drop(columns=['YEAR'])
y_train = y_train.drop(columns=['YEAR'])
y_test = y_test.drop(columns=['YEAR'])
X_train= scaler.fit_transform(X_train)
X_test= scaler.transform(X_test)





In [34]:
xgb_r = XGBRegressor(n_estimators=1000, random_state=42)
xgb_r.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=42, ...)

In [35]:


y_pred = xgb_r.predict(X_test)
print(f'R Squared Score: {r2_score(y_pred, y_test)}')
print({mean_squared_error(y_test, y_pred)})
#y_pred
y_test

R Squared Score: -0.10629082444381344
{1278.1323924093033}


array([[-0.1350798 ],
       [-0.13969327],
       [-0.13969327],
       ...,
       [ 1.41255486],
       [ 0.09134931],
       [ 0.0033243 ]])

In [32]:
lsv = LinearSVR(verbose= 1, max_iter= 4000)
lsv.fit(X_train,np.ravel(y_train))

c:\Users\John\anaconda3\envs\time_ser1\lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


[LibLinear]{1673.6634831409194}


c:\Users\John\anaconda3\envs\time_ser1\lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [ ]:
np.ravel(y_train)

In [33]:

y_pred = lsv.predict(X_test)
print(f'R Squared Score: {r2_score(y_pred, np.ravel(y_test))}')
print({mean_squared_error(y_test, y_pred)})

R Squared Score: -0.048902853706501936
{1673.7522032940044}


In [19]:
#print(y_pred[:5])
#print(y_test[0:5])
print(X_test[:5])
print(X_train[:5])

[[-0.00886123 -0.00542624 -0.01799946 ...  1.         -2.5633199
  -2.46857469]
 [-0.00886123 -0.00542624 -0.01799946 ...  1.         -2.95597674
  -2.46857469]
 [-0.00886123 -0.00542624 -0.01799946 ...  1.         -2.95597674
  -2.46857469]
 [-0.00886123 -0.00542624 -0.01799946 ...  1.         -2.95597674
  -2.46857469]
 [-0.00886123 -0.00542624 -0.01799946 ...  1.         -2.95597674
  -2.46857469]]
[[-0.00886123 -0.00542624 -0.01799946 ...  0.         -1.77800623
  -1.29408334]
 [-0.00886123 -0.00542624 -0.01799946 ...  0.         -1.77800623
  -1.29408334]
 [-0.00886123 -0.00542624 -0.01799946 ...  0.         -1.77800623
  -1.29408334]
 [-0.00886123 -0.00542624 -0.01799946 ...  0.         -1.77800623
  -1.29408334]
 [-0.00886123 -0.00542624 -0.01799946 ...  0.         -1.77800623
  -1.29408334]]


In [21]:
rf = RandomForestRegressor(n_estimators=300,max_depth=10, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

c:\Users\John\anaconda3\envs\time_ser1\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor(max_depth=10, n_estimators=300, n_jobs=-1,
                      random_state=42)

In [29]:
y_pred = rf.predict(X_test)
print(f'R Squared Score: {r2_score(y_pred, np.ravel(y_test))}')

print({mean_squared_error(y_test, y_pred)})


R Squared Score: -0.2983500364901355
{1607.378042198023}


In [37]:
# Various hyper-parameters to tune
xgb1 = XGBRegressor()
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500, 1000],
              'random_state': [42]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(X_train,
         y_train)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

Fitting 2 folds for each of 18 candidates, totalling 36 fits
[15:04:56] WARNING: D:\bld\xgboost-split_1691196841096\work\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[15:04:56] WARNING: D:\bld\xgboost-split_1691196841096\work\src\learner.cc:767: 
Parameters: { "silent" } are not used.

0.1897132658324035
{'colsample_bytree': 0.7, 'learning_rate': 0.03, 'max_depth': 5, 'min_child_weight': 4, 'n_estimators': 1000, 'nthread': 4, 'objective': 'reg:linear', 'random_state': 42, 'silent': 1, 'subsample': 0.7}


In [38]:
# Various hyper-parameters to tune
xgb1 = XGBRegressor()
parameters = {#'nthread':[4], #when use hyperthread, xgboost may become slower
              #'objective':['reg:linear'],
              #'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7],
              #'min_child_weight': [4],
              #'silent': [1],
              #'subsample': [0.7],
              #'colsample_bytree': [0.7],
              'n_estimators': [500, 1000, 1500, 2000],
              'random_state': [42]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = -1,
                        verbose=True)

xgb_grid.fit(X_train,
         y_train)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

Fitting 2 folds for each of 12 candidates, totalling 24 fits
0.1853560811870199
{'max_depth': 7, 'n_estimators': 500, 'random_state': 42}


In [39]:
#Lets try the best model accoridng gridsearchcv
# {'max_depth': 7, 'n_estimators': 500, 'random_state': 42}

xgb_r = XGBRegressor( max_depth= 7, n_estimators= 500, random_state= 42)
xgb_r.fit(X_train,y_train)

In [ ]:
y_pred = xgb_r.predict(X_test)
print(f'R Squared Score: {r2_score(y_pred, y_test)}')
print({mean_squared_error(y_test, y_pred)})